In [1]:
import os
os.getenv("SLURM_TMPDIR")

'/scratch/local/64463386'

In [3]:
os.chdir(os.getenv('SLURM_TMPDIR'))

# Restore non-JvM cubes from distributed data

We used JvM (https://ui.adsabs.harvard.edu/abs/1995AJ....110.2037J/abstract) correction to produce the distributed data cubes.  

In the JvM correction process, the residuals are suppressed by a factor set by the ratio of the dirty beam within the first null to the clean beam.

If you want to restore the residuals to their original values, this script demonstrates how, starting with the data download:

## Download data:
Download some data using the script at https://gist.github.com/keflavich/46cbf66a4c5ba7a9d64753acdfeeb036

In [4]:
# An API key on dataverse is required
with open(os.path.expanduser('~/.almaimf_dataverse_token'), 'r') as fh:
    api_key = token = fh.read().strip()

In [5]:
field_id = 'G353.41_B6_spw1_12M_spw1'

In [7]:
import requests, os

dataverse_server='https://dataverse.harvard.edu'

suffix = {'model': 'YWW5BY',
          'pb': 'RBS6KT',
          'cont': 'SF09HK',
          'cube': 'CJ3YXU'}

for doiend in suffix.values():
    persistentId = f'doi:10.7910/DVN/{doiend}'

    url_list = f'{dataverse_server}/api/datasets/:persistentId/versions/:draft/files?key={api_key}&persistentId={persistentId}'
    filelist_resp = requests.get(url_list)
    file_metadata = filelist_resp.json()

    # loop through *all files* and download them if they're not already downloaded
    for fm in file_metadata['data']:
        if field_id in fm['dataFile']['filename'] and not os.path.exists(fm['dataFile']['filename']):
            print("Downloading ", fm['dataFile']['filename'])
            resp = requests.get(f"{dataverse_server}/api/access/datafile/{fm['dataFile']['id']}", stream=True,
                                params={'key': api_key},
                                auth=(api_key, ''), )
            resp.raise_for_status()
            with open(fm['dataFile']['filename'], 'wb') as fh:
                fh.write(resp.content)

In [8]:
from spectral_cube import SpectralCube

In [37]:
cube = SpectralCube.read('G353.41_B6_spw1_12M_spw1.JvM.image.pbcor.statcont.contsub.fits')
cube.allow_huge_operations = True
cube

SpectralCube with shape=(959, 823, 819) and unit=Jy / beam:
 n_x:    819  type_x: RA---SIN  unit_x: deg    range:   262.586003 deg:  262.632997 deg
 n_y:    823  type_y: DEC--SIN  unit_y: deg    range:   -34.716545 deg:  -34.677728 deg
 n_s:    959  type_s: FREQ      unit_s: Hz     range: 217045473750.146 Hz:217279338345.200 Hz

In [40]:
for com in cube.header['COMMENT']:
    if 'JvM_epsilon_median' in com:
        JvM_Factor = float(com.split("=")[-1])
JvM_Factor

0.7887069317340462

In [25]:
from astropy.io import fits
from astropy import units as u

In [20]:
pb = fits.open(f'{field_id}.flatpb.fits')

In [21]:
non_pb_cube = cube * pb[0].data
non_pb_cube.write(f'{field_id}.JvM.image.statcont.contsub.fits')

In [26]:
cont = fits.open(f'{field_id}.JvM.image.pbcor.statcont.cont.fits')

In [30]:
non_contsub_cube = cube + cont[0].data * u.Unit(cont[0].header['BUNIT'])
non_contsub_cube.write(f'{field_id}.JvM.image.pbcor.fits', overwrite=True)
non_contsub_non_pb_cube = non_contsub_cube * pb[0].data
non_contsub_non_pb_cube.write(f'{field_id}.JvM.image.fits', overwrite=True)

In [42]:
modcube = SpectralCube.read(f'{field_id}.model.minimized.fits.gz')
modcube.allow_huge_operations = True
modcube

SpectralCube with shape=(959, 823, 819) and unit=Jy / pix:
 n_x:    819  type_x: RA---SIN  unit_x: deg    range:   262.586003 deg:  262.632997 deg
 n_y:    823  type_y: DEC--SIN  unit_y: deg    range:   -34.716545 deg:  -34.677728 deg
 n_s:    959  type_s: FREQ      unit_s: Hz     range: 217045473750.146 Hz:217279338345.200 Hz

In [43]:
import radio_beam

In [ ]:
modcube_with_deltabeam = modcube.with_beam(radio_beam.Beam(1e-6*u.arcsec))
residual = (non_contsub_non_pb_cube - modcube_with_deltabeam.convolve_to(cube.beam) * u.pixel/u.beam) / JvM_factor
residual.write(f'{field_id}.residual.fits')